In [1]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.5 MB/s eta 0:00:00


In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, sum

# Инициализация сессии
spark = SparkSession.builder.appName("Logs Analysis").master("local[*]").getOrCreate()

# Формирование датафрейма из csv файла
df = spark.read.csv("web_server_logs.csv", header=True, inferSchema=True)

# Группировка данных по IP и рассчет количества запросов для каждого IP, выведено 10 самых активных IP
top_ips = df.groupBy("ip").count().withColumnRenamed("count","request_count").orderBy(col("request_count").desc()).limit(10)
print("Top 10 active IP addresses:")
top_ips.show()

# Группировка данных по HTTP-методам и рассчет количества запросов для каждого метода
method_counts = df.groupBy("method").count().withColumnRenamed("count", "method_count")
print("Request count by HTTP method:")
method_counts.show()

# Фильтрация и рассчет количества запросов с кодом ответа 404
filtered_df = df.filter(col("response_code") == 404).count()
print(f"Number of 404 response codes: {filtered_df}")

# Группировка данных по дате и суммирование размеров ответов, отсортированных по дате
total_size = df.withColumn("date", to_date(col("timestamp"), "yyyy-MM-dd")).groupBy("date").agg(sum(col("response_size")).alias("total_response_size")).orderBy(col("date"))
print("Total respose size by day:")
total_size.show()

# Завершение сессии
spark.stop()

Top 10 active IP addresses:
+---------------+-------------+
|             ip|request_count|
+---------------+-------------+
| 90.103.226.185|            2|
|220.161.163.231|            1|
|196.134.147.245|            1|
|   89.8.111.227|            1|
|   86.51.125.84|            1|
| 223.209.22.127|            1|
| 166.164.100.85|            1|
|  32.221.40.158|            1|
|   24.236.53.56|            1|
|195.115.131.147|            1|
+---------------+-------------+

Request count by HTTP method:
+------+------------+
|method|method_count|
+------+------------+
|  POST|       24832|
|DELETE|       25277|
|   PUT|       24807|
|   GET|       25084|
+------+------------+

Number of 404 response codes: 24922
Total respose size by day:
+----------+-------------------+
|      date|total_response_size|
+----------+-------------------+
|2024-01-01|            1760344|
|2024-01-02|            1733523|
|2024-01-03|            1737273|
|2024-01-04|            1709376|
|2024-01-05|          